In [9]:
from sklearn.neighbors import KNeighborsClassifier

The core idea of KNN is to compute the distance between the query point and all other points in the dataset.

The most common distance metric is Euclidean distance, but others like Manhattan or Minkowski distances can also be used.

The algorithm then ranks all points in the dataset based on how close they are to the query point.

The algorithm selects the k closest data points (neighbors) based on the distance metric.

The labels of the k nearest neighbors are considered.

Majority voting is performed: the class with the most votes is assigned to the query point.


In [18]:
import cv2
import pickle
import numpy as np
import os
import csv
import time 
from datetime import datetime
from win32com.client import Dispatch  #for speaking purpose

def speak(str1):
    speak = Dispatch(("SAPI.spvoice"))
    speak.Speak(str1)

# Capturing video from the internal webcam 0 denotes internal peripheral
video = cv2.VideoCapture(0)

# Loading the pre-trained face detection model (Haar Cascade)
facedetect = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
with open('data/names.pkl','rb') as f:
    LABELS = pickle.load(f)

with open('data/faces_data.pkl','rb') as f:
    FACES = pickle.load(f)

knn = KNeighborsClassifier(n_neighbors = 5) #based on 5 nbd it decides class
knn.fit(FACES, LABELS)

COL_NAMES = ['NAME','TIME']

while True:
    ret,frame=video.read()
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces=facedetect.detectMultiScale(gray, 1.3 ,5)
    for (x,y,w,h) in faces:
        crop_img=frame[y:y+h, x:x+w, :]
        resized_img=cv2.resize(crop_img, (50,50)).flatten().reshape(1,-1)
        output=knn.predict(resized_img)
        ts = time.time() #pass it to get date format
        date = datetime.fromtimestamp(ts).strftime("%d-%m-%Y")
        timestamp = datetime.fromtimestamp(ts).strftime("%H:%M:%S")

        #Checking is there any attendance sheet already exists on current date
        exist = os.path.isfile("Attendance/Attendance_"+ date +".csv")
        
        # Display the predicted label on the frame
        # Position the text just below the rectangle
        text_position_x = x + w - 50
        text_position_y = y + h + 25  # Slight padding of 20 pixels below the rectangle
        
        # Display the predicted label just below the rectangle
        cv2.putText(frame, str(output[0]), (text_position_x, text_position_y), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 4)

        attendance = [str(output[0]),str(timestamp)]    
    cv2.imshow("frame",frame) #window_name, material "to view"
    k = cv2.waitKey(1) #wait for input press p 
    if k == ord('p'):        
        if exist: #Current date attendance sheet already exist
            if output[0] not in name_list:
                with open("Attendance/Attendance_"+ date +".csv","+a")as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow(attendance)
                    
                    name_list.append(output[0])
                    speak("Attendance taken" + str(output[0]))
                    time.sleep(2)
                csvfile.close()

            else:
                if str(output[0]) != " ":
                    speak(str(output[0]+"Your Attendance is already taken"))
                    time.sleep(2)
                else:
                    speak("kindly show your face properly")
        else:
            with open("Attendance/Attendance_"+ date +".csv","+a")as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(COL_NAMES)
                writer.writerow(attendance)
                name_list = [output[0]]
                
                speak("Attendance taken" + str(output[0]))
                time.sleep(2)
            csvfile.close()
        
    if k == ord('q'): # press q to quite
        break

video.release()
cv2.destroyAllWindows()
